# Pi formula Secuential

In [7]:
%%writefile PisecCOLLAB.c
#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>

#define ITERATIONS 2e9

void Pileibnitz(double *pi){
    for (int i=0; i<ITERATIONS;i++){
        *pi += (double) 1.0/(i*2+1)*4.0;
        i++;
        *pi -= (double) 1.0/(i*2+1)*4.0;
    }
}

void main(){
    FILE *fo;
    fo = fopen ("PiOPMCOLLAB.txt", "a");
    double pi=0.0;
    struct timeval tval_before, tval_after, tval_result;
    gettimeofday(&tval_before, NULL);

    Pileibnitz(&pi);

    gettimeofday(&tval_after, NULL); 
    timersub(&tval_after, &tval_before, &tval_result);
    long int seconds = (long int)tval_result.tv_sec;
    long int useconds = (long int)tval_result.tv_usec;


    fprintf(fo, "1 %ld.%06ld\n", seconds, useconds);
    fclose(fo);

    printf("Pi calculated with %f iteractions \n With a value of %f \n Executed in %ld.%06ld seconds", ITERATIONS, pi, seconds, useconds); //time execution calc command for linux
}

Overwriting PisecCOLLAB.c


In [8]:
%%shell
gcc PisecCOLLAB.c -o PisecCOLLAB -fopenmp
./PisecCOLLAB

Pi calculated with 2000000000.000000 iteractions 
 With a value of 3.141593 
 Executed in 20.817805 seconds

# Pi value with 2,3,4,8,16 threads

In [17]:
%%writefile PiMPThreads.c
#include <stdlib.h>
#include <stdio.h>
#include <omp.h>
#include <sys/time.h>

#define ITERATIONS 2e9
#define THREADS 16
int range = ITERATIONS/THREADS;


void Pileibnitz(double *picalc, int thread_num){
    int init = range * thread_num;
    int end = range * (thread_num+1);
    for (int i=init; i<end;i++){
        *(picalc+thread_num) += (double) (4.0/(i*2+1));
        i++;
        *(picalc+thread_num) -= (double) (4.0/(i*2+1));
    }
    //printf("\nRange of thread %i begin in %i and ends at %i\n", thread_num, init, end);
}

void main(){
    FILE *fo;
    fo = fopen ("PiOPMCOLLAB.txt", "a");
    double pi[THREADS];
    struct timeval tval_before, tval_after, tval_result;
    gettimeofday(&tval_before, NULL);

    for(int i=0; i<THREADS; i++){
        pi[i]=0;
    }


    #pragma omp parallel num_threads(THREADS)
    {
        int ID = omp_get_thread_num();
        Pileibnitz(pi, ID);
    }
    //printf("\n Thread %i with pi value of %2.12f\n", 0, pi[0]);
    for(int j=1; j<THREADS; j++){
        pi[0] += pi[j];
        //printf("\n Thread %i with pi value of %2.12f\n", j, pi[j]);
    }

    gettimeofday(&tval_after, NULL); 
    timersub(&tval_after, &tval_before, &tval_result);
    long int seconds = (long int)tval_result.tv_sec;
    long int useconds = (long int)tval_result.tv_usec;
    
    fprintf(fo, "%i %ld.%06ld\n", THREADS ,seconds, useconds);
    fclose(fo);

    printf("Pi calculated with %f iteractions \n With a value of %2.12f \n Executed in %ld.%06ld seconds", ITERATIONS, pi[0], seconds, useconds); 
}

Overwriting PiMPThreads.c


In [18]:
%%shell
gcc PiMPThreads.c -o PiMPThreads -fopenmp
./PiMPThreads

Pi calculated with 2000000000.000000 iteractions 
 With a value of 3.141592658507 
 Executed in 8.345906 seconds